In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/인지프

/content/drive/MyDrive/인지프


In [ ]:
# import package

# model
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch import optim

# dataset and transformation
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import models

# display images
from torchvision import utils
import matplotlib.pyplot as plt
%matplotlib inline

# utils
import numpy as np
from glob import glob
import os
from tqdm import tqdm

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

Using cuda for inference


In [ ]:
# train test split

import csv
import shutil

train_list = []
test_list = []

with open('train.csv', 'r', encoding='utf-8') as f:
  rdr = csv.reader(f)
  for line in rdr:
      train_list.append(line[1])

with open('test.csv', 'r', encoding='utf-8') as f:
  rdr = csv.reader(f)
  for line in rdr:
      test_list.append(line[1])

# full_data_path = './data/'

train_list = train_list[1:]
test_list = test_list[1:]

print(f'csv len: {len(test_list) + len(train_list)}')
# print(f'full data len: {len(os.listdir("./data"))}')

# for test in tqdm(test_list):
#   if os.path.exists('./data/' + test + '.jpg'):  
#     img_path = glob('./data/' + test + '.jpg')[0]
#     shutil.copyfile(img_path, './test/' + test + '.jpg')
  
#   else:
#     print(test)

# for train in tqdm(train_list):
#   if os.path.exists('./data/' + train + '.jpg'):  
#     img_path = glob('./data/' + train + '.jpg')[0]
#     shutil.copyfile(img_path, './train/' + train + '.jpg')
#   else:
#     print(train)

print(f'test len: {len(os.listdir("./test"))}')
print(f'train len: {len(os.listdir("./train"))}')

csv len: 15287
test len: 1529
train len: 13758


In [ ]:
!pip install efficientnet_pytorch


from efficientnet_pytorch import EfficientNet
efficientnet = EfficientNet.from_pretrained('efficientnet-b0')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Loaded pretrained weights for efficientnet-b0


In [ ]:
from PIL import Image
from torch.utils.data import Dataset, DataLoader


class SampleDataset(Dataset):
    def __init__(self, phase='test'):
        # self.path = './sample_data'
        self.phase = phase
        
        self.img_list = glob(self.phase + '/*')

        self.transform = transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Pad(padding=(0, 140), padding_mode='reflect'),
                               transforms.Resize((224, 224))
                         ])
    
    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self, idx):
        img_path = self.img_list[idx]

        img = Image.open(img_path)
        img = self.transform(img)

        vid = img_path.split('/')[-1][:-4]
        # print(vid)

        return img, vid

In [ ]:
# Prepare sample input data.

batch_size = 64

test_dataset = SampleDataset(phase='test')
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

train_dataset = SampleDataset(phase='train')
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import pickle

save_data = {}


with torch.no_grad():
  for img, vid in tqdm(test_dataloader):
    features = efficientnet.extract_features(img)
    # print(features.shape)

    for b in range(batch_size):
      if b < len(vid):
        save_data[vid[b]] = features[b]

print(f'test len: {len(test_list)}')
print(f'data len: {len(save_data)}')

with open('test.txt', 'wb') as f:
  pickle.dump(save_data, f, pickle.HIGHEST_PROTOCOL)

NameError: ignored

In [ ]:
save_data = {}

with torch.no_grad():
  for img, vid in tqdm(train_dataloader):
    
    features = efficientnet.extract_features(img)
    # print(features.shape)

    for b in range(batch_size):
      if b < len(vid):
        save_data[vid[b]] = features[b]

print(f'train len: {len(train_list)}')
print(f'data len: {len(save_data)}')

with open('train.pickle', 'wb') as f:
  pickle.dump(save_data, f, pickle.HIGHEST_PROTOCOL)

데이터 로더 안쓰는 버전

In [ ]:
# # without dataloader ver

# from PIL import Image
# import pickle

# save_data = {}

# transform = transform=transforms.Compose([
#                                transforms.ToTensor(),
#                                transforms.Pad(padding=(0, 140), padding_mode='reflect'),
#                                transforms.Resize((224, 224))
#                       ])

# for vid in tqdm(test_list):
#   img_path = './test/' + vid + '.jpg'

#   img = Image.open(img_path)
#   img = transform(img).unsqueeze(0)

#   # print(img.shape)

#   features = efficientnet.extract_features(img)
#   # print(features.shape)

#   save_data[vid] = features

# with open('test.pickle', 'wb') as f:
#   pickle.dump(save_data, f, pickle.HIGHEST_PROTOCOL)